# XPath Lesson

The object of this lesson is to introduce XPath syntax and show how it is used within the LXML Python package.  XPath is a powerful query language for XML data structures.  Many systems support XPath queries, including the Oxygen XML editor and several Python packages.  This lesson will focus on how it can be used within Python, but the core XPath content should be relevant enough you to follow along using a package or tool of your choice.  There will be two distinct sections:  (briefly) how to run XPath queries in Python and (a bit longer) how XPath queries are written and approached.  You may choose to follow along with the second section using whichever framework you are comfortable in.  I would recommend Oxygen XML editor if you have access to it.

The scope of this lession will also be limited to just getting started and getting exposed to the essential syntax elements of XPath.  This includes, reading in a set of XML files, running XPath queries on them, and outputting the results.  XSLT or writing out XML files will not be covered, as they belong in separate lessons.

## What you will need:

* a set of XML files
* a computer with Python or another platform to execute XPath queries
* at a minimum, a working comfort of Python
* to use Pip
* have LXML installed for your Python instance

## Why XPath and why not a regular expression?

As stated above, XPath is a query language for working on XML trees.  Many tutorials, usually those teasing at web scraping tasks, will show how a regular expression may be used to extract data out of XML data structures.  This is not an impossible task, depending on the situation, but using a regex in such a situation can pose a dangerous and limited path.  XPath is designed to run queries on XML using an XML-based syntax language.

Let's dwell on this query language/data structure relationship for a moment:

* Regular expressions are designed to work at the level of individual characters
* XPath is designed to work at the level of XML elements
* SQL is designed to work at the level database tables

While it may be tempting to try and make regular expressions work on an XML file, it can be dangerous because the structure of the raw text is nearly meaningless within the perspective of XML.  Let's look at an example XML file:

``` XML
<book>
    <author>Human, A.</author>
    <title>This is not a book</title>
</book>
```

In this very limited example, a regular expression could easily catch the text between the `author` tag, but rarely do we have such simple XML.  Let's look at what happens when complexity is added.

In [18]:
text = ["""<book>
    <author>Human, A.</author>
    <title>This is not a book</title>
</book>
""" ,

"""
<book>
    <author>Human, A.</author>
    <author>Human, Another.</author>
    <title>This is not a book</title>
</book>
""" ,

"""
<book><author>Human, A.</author><title>This is not a book</title></book>
""" ]

In [2]:
import re

for t in text:
    print re.findall(r'<author>(.+)<\/author>', t)

['Human, A.']
['Human, A.', 'Human, Another.']
['Human, A.']


This regex is holding strong...

In [3]:
book = """
<book>
    <author kind = "editor">Human, A.</author>
    <author kind    =  "contributor">Human, Another.</author>
    <title>This is not a book</title>
</book>
"""

re.findall(r'<author>(.+)<\/author>', book)

[]

But regex treats all characters in the text as meaninful, when this is not true for the data structure we are working with.  Imagine writing the regex for when you're looking at multiple attributes that can appear in any order.  Sometimes there are situations where regex is just fine, but XPath should be your default choice for handling XML data.

# Sample workflow

Several packages within Python have support for XPath, which is the beauty of utilizing this tool.  This tutorial will feature the LXML package.  [LXML](http://lxml.de/) has their own installation directions, which I will refer you to externally: http://lxml.de/installation.html.  However, it is supported via `pip` and `conda`.

In the spirit of a Programming Historial style, let's start with just a sample workflow.

Outline of this task:

* read in source XML files
* parse via LXML
* use an XPath statement to extract some information
* assemble and dump out that informatin

More specifically, we'll be using the 2013 Digital Humanities abstracts.  These files are formatted in TEI and available at the conference website: http://dh2013.unl.edu/abstracts/.  There are two versions: an corpus of all the abstracts and the individual files.  This is not a tutorial on reading through multiple files, so this activity will be included but not expanded on.

Specific tasks:

* read in the TEI corpus
* extract the ID, title, and type of each abstract
* write out those results to a CSV

## Setting up

In [1]:
from lxml import etree

# 

In [2]:
source_xml = 'DH 2013 final_xml/ab-130.xml'

with open(source_xml, 'rt') as read_in:
    raw_text = read_in.read() # read in the basic text, the file is now a string
    
tree = etree.fromstring(raw_text) # parse the string of text

At this point, we've read in the plain text version of our XML file and parsed that using the `etree` library from LXML.  Let's play with this for a bit.  LXML has pretty full documentation about how to use `etree` here: http://lxml.de/tutorial.html#using-xpath-to-find-text  We're going to focus on the bare bones navigation and text extraction.

In [3]:
print tree

<Element {http://www.tei-c.org/ns/1.0}TEI at 0x1051b3368>


Once the file is parsed each element becomes an object we can act on.  Very roughly, we can manually inspect the file contents and know that the structure is (with `...` meaning that some code was snipped:

``` XML
<TEI>
    <teiHeader>
        <titleStmt>
            <title>...</title>
        </titleStmt>
    ...
    </teiHeader>
    ...
    <text type = '...'>
        <body>
        ...
        </body>
    </text>
</TEI>
```

The `{http://www.tei-c.org/ns/1.0}TEI` notation is also important because it tells you that the element is being parsed as part of a namespace.  If we tried to just access `TEI` it would fail because the parser is expecting `{http://www.tei-c.org/ns/1.0}TEI` to appear.  Let's look at a basic XPath command to just extract the title text out of this individual abstract.  We need to include the namespace for TEI in the process.

In [4]:
print tree.xpath('//title/text()') # returns nothing as we expect.

[]


Note that even though this is a valid XPath statement (even though we haven't introduced it yet, so trust me), it is returning nothing.  When the element `<TEI>` is being displayed as `{http://www.tei-c.org/ns/1.0}TEI` we know now that we will need to include a namespace declaration within our XPath statements.

Ignoring a bit of the Python-specific syntax here for a moment, the LXML tools that we're using for XPath statements does have the capacity to delcare the namespaces and associated names.

Example:  `your_tree.xpath('//namespace_name:element_name/text()', namespaces = {'namespace_name': 'schema_location_url')`

The bits that you need to change are the `namespace_name` value, which is just what you want to call the namespace within your XPath statemtents.  This value just needs to be unique and something you don't mind typing in a bunch.  The second is `schema_location_url`, which is the URL to the schema you are giving a name to.  This URL should be what you are seeing within the top of the XML document you are trying to query within.  After all, the parser won't know that an XML document has a namespace unless it is stated.  In this case we can look in the top of the document and see that it includes:  `<TEI xmlns="http://www.tei-c.org/ns/1.0" xml:id="ab-130">`

This is why the LXML parser is stating that `TEI` resides within the `{http://www.tei-c.org/ns/1.0}` namespace by returning: `{http://www.tei-c.org/ns/1.0}TEI`

Many times we'll have a long series of xpath statements to extract out all the information that we want, so we can store these namespace values as a variable so long as it works for all of them.  `{'tei': 'http://www.tei-c.org/ns/1.0'}` The key in this dictionary is a the name declaration.  I can name it anything in here, so long as it is unique (which is also required by virtue of this data structure being a dictionary).  The value is the URL to that schema.

While you can continue to use the syntax where you declare the namespace verbatim in every XPath function call, it can become unweidly and clutter your code.  Putting our Python eyes back on for a second, we can see that the namespace declaration object is simply a dictionary with the namespace declarations as they keys and the schema URL being the value.  We can then save this dictionary object into a separate variable as a separate part of our code we can call on at later points.

In [5]:
# create our namespace variable containing the namespace dictionary
# with the very creative variable name of `ns`
ns = {'tei': 'http://www.tei-c.org/ns/1.0'} 

# we can now replace the raw dictionary content 
# within the function call with our variable name
print tree.xpath('//tei:title/text()', namespaces = ns)

['A Comparative Study of Astronomical Clock towers in Europe and China based on their detailed 3D modeling']


At this point, we finally have all the pieces that we need to sucessfully create XPath statements that work on our XML document.  You may not be always working with documents that have required namespaces, so you may be able to skip this for your projects.

Let's turn our attention to what is happening within the statement used above: `//tei:title/text()`.  As a quick refresher for XML vocabulary, here's the naming taxonomy for a single element tag.

```XML 
<element_name attribute_name = 'attribute_value'>element_text</element_name>
```

The XPath language has methods of referring to these markup locations by name.  You'll also encounter metacharacters to indicate locations and internal function calls to perform tests and extract specific areas of text.  XPath should be considered within it's own box, separate and independent from the Python that is surrounding it.  Similar to using Regular Expresssions, this is an independent language that we are only using Python as a hook to perform it own a piece of text.

In the example XPath statement we've been using there are all three:  names, functions, and metacharacters.

* `//`: This means 'anywhere in the tree'
* `tei:`: This is the namesapce declaration that we defined as part of our xpath function call.  Recall that we gave http://www.tei-c.org/ns/1.0 the name `tei`.
* `title`: This is the element name.
* `/text()`: This is a function to return the text within the element returned from the xpath query.

Reframing our query into natural language, we might say "I want the element text content (`/text()`) from the `title` element (with the `tei` namespace), wherever it may be in the tree (`//`).

This means that there are two core quesitons that you need to answer within your XPath statement:

1. Where is the location?
2. What do you want from it?

# Defining XML locations

The location of nodes can be expressed both specifically by name or in relation to other nodes.  The core of defining these paths is the use of `/` and `//`.

* `/`: this stands for the 'root' element, which is the top-most element of the node tree
* `//`: this stands for 'anywhere

Let's go back to our TEI example and recall where the `title` element exists:

``` XML
<TEI>
    <teiHeader>
        <titleStmt>
            <title>...</title>
        </titleStmt>
    ...
    </teiHeader>
    ...
    <text type = '...'>
        <body>
        ...
        </body>
    </text>
</TEI>
```

There are actually multiple valid rules to define where this lives, and none are a bad way to do it aso long as it works.  But many of the alternatives are less direct.

Our previous example utilized this expression: `//tei:title`  Given that there is only one title within the XML file in question, this would be equivalent to `/tei:TEI/tei:teiHeader/tei:fileDesc/tei:titleStmt/tei:title` would be just as valid.  We can test these both and see that they return the same result:

In [6]:
ns = {'tei': 'http://www.tei-c.org/ns/1.0'} 

print tree.xpath('//tei:title/text()', namespaces = ns) 
print tree.xpath('/tei:TEI/tei:teiHeader/tei:fileDesc/tei:titleStmt/tei:title/text()', namespaces = ns)

# and something in between...

print tree.xpath('/tei:TEI/tei:teiHeader//tei:title/text()', namespaces = ns)

['A Comparative Study of Astronomical Clock towers in Europe and China based on their detailed 3D modeling']
['A Comparative Study of Astronomical Clock towers in Europe and China based on their detailed 3D modeling']
['A Comparative Study of Astronomical Clock towers in Europe and China based on their detailed 3D modeling']


Even though you provide a very specific path to a node, multiple values may be returned if there are multiple hits.

In [7]:
# so we can see that it is finding two author elements
print tree.xpath('/tei:TEI/tei:teiHeader/tei:fileDesc/tei:titleStmt/tei:author/tei:name', namespaces = ns)

[<Element {http://www.tei-c.org/ns/1.0}name at 0x1051b3950>, <Element {http://www.tei-c.org/ns/1.0}name at 0x1051b38c0>]


You may choose whichever style makes the most sense to your data.  Using the longer more specific path is usually the safest if you only want the values for a very specific node location, but there are times when the parent and ancestor nodes cannot be dependably predicted.  Depending on the schema and how controlled the content is, you may not be able to predict exactly where certain node values appear.  For example, if we wanted to extract the text out of all the `head` elements, we could start with this:

In [8]:
print len(tree.xpath('/tei:TEI/tei:text/tei:body/tei:div/tei:div/tei:head', namespaces = ns))

5


We can do a quick search within the raw XML file and find that there are 12 instances of `<head>`.  This specific XPath expression is failing because there are `<head>` elements at all levels within the `<body>` element.  This is where the magic of `//` comes in handy.

In [9]:
print len(tree.xpath('/tei:TEI/tei:text/tei:body//tei:head', namespaces = ns))
print len(tree.xpath('//tei:head', namespaces = ns))

12
12


In this specific example, we know the schema and can correctly predict that all `<head>`s will appear within the `<body>` node and no where else.  This should mean that `//tei:head` is appropriate and safe.  However, `/tei:TEI/tei:text/tei:body//tei:head` offers a nice balance between restriction and flexibility.

Another method of making a relative, but controlled selection is to find an interor element and select the node just above it.  This method will normally combine with a conditional checking method that we will be exploring in a later section, but we can use a brief HTML example here.

Taking a closer look at the TEI file that we have been working through, we can see in the user provided content that there are many `<div>` elements.  We may be in a situation where we would like to extract the text content out of all the `<div>`s that actually make up of the formatted content.  However, because several of the `<div>` elements enclose others, just running `//div` will select more than what we want.

Taking a closer look at the element structure can give us a clue about how to be more specific.

The `..` (yes, two periods in a row) notation allows you to make a selection 'move up' a node within your query.  This concept can be a bit tricky to explain, and is best illustrated by example.  Let's use a small piece of HTML to explore.  Say that for this given HTML snippit, we want to extract the element value from the `<p>` element from the `<div>` with the `<h1>`.  Okay, that's quite a convoluded way of stating it.  We could state that we wanted to select the `<p>` elements that are sibling to an `<h1>` within a `<div>`.  All of our previous queries had us looking further into a tree, presuming that each element that mattered in determining location was nested below the previous.  This does not work when the elements are siblings (children of the same node).

In [10]:
dot_dot_example = """
<div>
    <div>
        <h1>Things!</h1>
        <p>feelings about things.</p>
        <p>feeeeeelings</p>
        <p>nothing but feeeelings</p>
    </div>
    
    <div>
        <h2>References</h2>
        <p>[1]...</p>
        <p>[2]...</p>
    </div>
    
    <div>
        <h1>Things, but without p tags!</h1>
    </div>
</div>
"""

tree = etree.fromstring(dot_dot_example)

print tree.xpath('//div') # sees three div elements
print tree.xpath('//div/h1') # just to prove that this will find nothing
print tree.xpath('//div//p') # this will find the p tags that are sibling to the h2 as well
print tree.xpath('//div/h1/../p') # this is the one we want! let's get the text

print tree.xpath('//div/h1/../p/text()')

[<Element div at 0x1051b3c68>, <Element div at 0x1051b3cb0>, <Element div at 0x1051b3d40>, <Element div at 0x1051b3d88>]
[<Element h1 at 0x1051b3cf8>, <Element h1 at 0x1051b3d40>]
[<Element p at 0x1051b3cb0>, <Element p at 0x1051b3d40>, <Element p at 0x1051b3d88>, <Element p at 0x1051b3ef0>, <Element p at 0x1051b3e18>]
[<Element p at 0x1051b3cf8>, <Element p at 0x1051b3d40>, <Element p at 0x1051b3d88>]
['feelings about things.', 'feeeeeelings', 'nothing but feeeelings']


Let's break down what `//div/h1/../p/text()` is stating and why it is getting at what we want.  Every explicit node mentioned (or queried) in the statemnt must validate to something.  After all, these are paths, so when you mention a step, that step must be there or else it doesn't match.  You will see this if you attempt to work through things on your own.

1. Navigate, anywhere in the root, to a `<div>` element.
2. Look within the `<div>`s and find one where it has at least one `<h1>`.
3. Go up a node (e.g. back to the `<div>`).
4. Find just those divs (from the previous step) which have at least one `<p>`.
5. Return the text of the `<p>` elements selected from the previous step.

These processing subtleties should stress the importance of experimentation and testing when starting to use XPath queries. You will need to experiment with both to discover what is specific enough to catch what you need, but what might be so restrictive that you are losing elements that you are wanting to extract.  Working on large scale unstrucutred data, such as user created HTML content with no specific formatting, can cause you to inadvertently miss content you would have desired.  You may need to take an highly permissive route and run the content through other validation tests after extraction.

# Extracting content

add something here about the various functions, like /text(), //text(), /@stuff.

Now that you can select the nodes that you would like, you need to extract the content.

There are three core activities you'll want to do:

1. Select the text value out of an element's content.
2. Select all the text out of a node, however deep it may be in the tree.
3. Select the text content of an attribute value.

## Selecting the text out of a single element

## Selecting the text out of an entire node

## Selecting the text out of an attribute value

# Functions and conditional checking

bracket notationa and where the list of all the stuff is

# Applying an xpath query to a collection of XML files

the r language I just used there is noted.

glob, for, with, do xpath, throw into list, spit out to CSV. Bonus points if this gives me unicode errors.

# Where to go from here

W3C schools, the 'student's guide to XPath' book, etc.